### Q5 A.
I used sklearn's CountVectorizer and uses parameter vocabulary = dictionary.txt and I used TfidfVectorizer to get the tfidf of each documents and uses a function to get their respective feature names.

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

train_data = pd.read_csv('news-train-1.csv')
test_data = pd.read_csv('news-test.csv')

with open('dictionary.txt') as f:
	dic = f.read().splitlines()

train_data_text = train_data['Text']
vectorizer = TfidfVectorizer(
    stop_words='english', vocabulary=dic, analyzer='word', lowercase=True)

train_data_X = vectorizer.fit_transform(train_data_text)
feature_names = vectorizer.get_feature_names_out()
df = pd.DataFrame(train_data_X.toarray(), columns=feature_names)

categories = train_data['Category']
encoder = LabelEncoder()
encoded_cat = encoder.fit_transform(categories)

#### Q5 B
Based from Q3 and Q4, RandomForest yielded a better avg. accuracy score than XGBoost.
In Q3, the best n_estimator = 350. I will be using 'gini' which is the default criterion as my criterion based on previous result. Also, with cv=5

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

n_estimators = range(350, 1000, 15)
rf_param = {
    'n_estimators': n_estimators
}
rf_search = GridSearchCV(RandomForestClassifier(),rf_param, cv=5, verbose=10)
rf_search.fit(df.to_numpy(),encoded_cat)
print('Random forest best score: ', rf_search.best_score_)
print('Random forest best estimator: ', rf_search.best_estimator_)


Fitting 5 folds for each of 44 candidates, totalling 220 fits
[CV 1/5; 1/44] START n_estimators=350...........................................
[CV 1/5; 1/44] END ............n_estimators=350;, score=0.913 total time=   2.4s
[CV 2/5; 1/44] START n_estimators=350...........................................
[CV 2/5; 1/44] END ............n_estimators=350;, score=0.909 total time=   2.8s
[CV 3/5; 1/44] START n_estimators=350...........................................
[CV 3/5; 1/44] END ............n_estimators=350;, score=0.940 total time=   1.9s
[CV 4/5; 1/44] START n_estimators=350...........................................
[CV 4/5; 1/44] END ............n_estimators=350;, score=0.940 total time=   1.8s
[CV 5/5; 1/44] START n_estimators=350...........................................
[CV 5/5; 1/44] END ............n_estimators=350;, score=0.950 total time=   1.8s
[CV 1/5; 2/44] START n_estimators=365...........................................
[CV 1/5; 2/44] END ............n_estimators=365

In [3]:
rf_final = RandomForestClassifier(
	n_estimators=620,
	max_features='log2'
)
test_data_text = test_data['Text']

test_data_X = vectorizer.fit_transform(test_data_text)
test_feature_names = vectorizer.get_feature_names_out()
test_df = pd.DataFrame(test_data_X.toarray(), columns=test_feature_names)
test_df = test_df.fillna(value=0)
rf_final.fit(df,encoded_cat)

rf_pred = rf_final.predict(test_df)
rf_pred_label = encoder.inverse_transform(rf_pred)
test_data['Category'] = rf_pred_label
test_data[['ArticleId', 'Category']].to_csv('labels.csv', index=False, header=False)